EJERCICIO 1

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

# Creación de la sesión de Spark
spark = SparkSession.builder \
        .appName("Streaming WordCount") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .config("spark.sql.shuffle.partitions", 3) \
        .getOrCreate()

# Creamos un flujo de escucha sobre netcat en localhost:9999, mediante readStream
lineasDF = spark.readStream \
        .format("socket") \
        .option("host", "localhost") \
        .option("port", "9999") \
        .load()

# Leemos las líneas, las dividimos en palabras y realizamos la agrupación con count (transformación)
palabrasDF = lineasDF.select(explode(split(lineasDF.value, ' ')).alias('palabra'))
cantidadDF = palabrasDF.groupBy("palabra").count()

# Persistencia de los resultados mediante writeStream
wordCountQuery = cantidadDF.writeStream \
    .queryName("Caso1WordCount") \
    .format("console") \
    .outputMode("complete") \
    .start()

# Dejamos Spark a la escucha
wordCountQuery.awaitTermination()

25/02/20 18:41:58 WARN Utils: Your hostname, iadb-12 resolves to a loopback address: 127.0.1.1; using 172.20.104.44 instead (on interface wlp44s0f0)
25/02/20 18:41:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/20 18:41:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/20 18:42:00 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
25/02/20 18:42:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-78939943-fa50-4c68-ab2e-350e37b6e454. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----+
|palabra|count|
+-------+-----+
+-------+-----+



25/02/20 18:42:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----+
| palabra|count|
+--------+-----+
|funciona|    1|
|    bien|    1|
|    aqui|    1|
+--------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------+-----+
| palabra|count|
+--------+-----+
|funciona|    1|
|    bien|    1|
|    aqui|    1|
|       Ñ|    1|
+--------+-----+



25/02/20 18:43:28 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=66>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exc

Py4JError: An error occurred while calling o51.awaitTermination

EJERCICIO 2

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr


spark = SparkSession \
       .builder \
       .appName("Streaming de Ficheros") \
       .master("local[*]") \
       .config("spark.streaming.stopGracefullyOnShutdown", "true") \
       .config("spark.sql.shuffle.partitions", 3) \
       .config("spark.sql.streaming.schemaInference", "true") \
       .getOrCreate()


raw_df = spark.readStream \
    .format("json") \
    .option("path", "Invoices") \
    .option("maxFilesPerTrigger", 1) \
    .option("cleanSource", "delete") \
    .load()

raw_df.printSchema()


explode_df = raw_df.selectExpr("InvoiceNumber", "CreatedTime", "StoreID","PosID", "CustomerType",
                                "PaymentMethod", "DeliveryType", "explode(InvoiceLineItems) as LineItem")
explode_df.printSchema()


limpio_df = explode_df \
   .withColumn("ItemCode", expr("LineItem.ItemCode")) \
   .withColumn("ItemDescription", expr("LineItem.ItemDescription")) \
   .withColumn("ItemPrice", expr("LineItem.ItemPrice")) \
   .withColumn("ItemQty", expr("LineItem.ItemQty")) \
   .withColumn("TotalValue", expr("LineItem.TotalValue")) \
   .drop("LineItem")
limpio_df.printSchema()


facturaWriterQuery = limpio_df.writeStream \
   .format("json") \
   .queryName("Facturas Writer") \
   .outputMode("append") \
   .option("path", "Salida") \
   .option("checkpointLocation", "chk-point-dir-caso2") \
   .trigger(processingTime="1 minute") \
   .start()


facturaWriterQuery.explain() # muestra una explicación detalla del plan de ejecución
facturaWriterQuery.recentProgress # muestra una lista de los últimos progresos de la consulta
facturaWriterQuery.lastProgress # muestra el último progreso

25/02/20 18:46:31 WARN Utils: Your hostname, iadb-12 resolves to a loopback address: 127.0.1.1; using 172.20.104.44 instead (on interface wlp44s0f0)
25/02/20 18:46:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/20 18:46:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- CESS: double (nullable = true)
 |-- CGST: double (nullable = true)
 |-- CashierID: string (nullable = true)
 |-- CreatedTime: long (nullable = true)
 |-- CustomerCardNo: string (nullable = true)
 |-- CustomerType: string (nullable = true)
 |-- DeliveryAddress: struct (nullable = true)
 |    |-- AddressLine: string (nullable = true)
 |    |-- City: string (nullable = true)
 |    |-- ContactNumber: string (nullable = true)
 |    |-- PinCode: string (nullable = true)
 |    |-- State: string (nullable = true)
 |-- DeliveryType: string (nullable = true)
 |-- InvoiceLineItems: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ItemCode: string (nullable = true)
 |    |    |-- ItemDescription: string (nullable = true)
 |    |    |-- ItemPrice: double (nullable = true)
 |    |    |-- ItemQty: long (nullable = true)
 |    |    |-- TotalValue: double (nullable = true)
 |-- InvoiceNumber: string (nullable = true)
 |-- NumberOfItems: long (nullable = t

25/02/20 18:46:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


25/02/20 18:46:48 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


EJERCICIO 3

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Kafka Streaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4") \
    .master("local[*]") \
    .getOrCreate()

kafkaDFS = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "facturas") \
    .option("startingOffsets", "earliest") \
    .load()

25/02/20 19:35:56 WARN Utils: Your hostname, iadb-12 resolves to a loopback address: 127.0.1.1; using 172.20.104.106 instead (on interface enp45s0)
25/02/20 19:35:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/iabd/.ivy2/cache
The jars for the packages stored in: /home/iabd/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-88cc1915-b211-44ca-af2b-aa6497b88a44;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 226ms :: artifacts dl 10ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#

In [2]:
kafkaDFS.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [3]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType
esquema = StructType([
    StructField("InvoiceNumber", StringType()),
    StructField("CreatedTime", LongType()),
    StructField("StoreID", StringType()),
    StructField("PosID", StringType()),
    StructField("CashierID", StringType()),
    StructField("CustomerType", StringType()),
    StructField("CustomerCardNo", StringType()),
    StructField("TotalAmount", DoubleType()),
    StructField("NumberOfItems", IntegerType()),
    StructField("PaymentMethod", StringType()),
    StructField("CGST", DoubleType()),
    StructField("SGST", DoubleType()),
    StructField("CESS", DoubleType()),
    StructField("DeliveryType", StringType()),
    StructField("DeliveryAddress", StructType([
        StructField("AddressLine", StringType()),
        StructField("City", StringType()),
        StructField("State", StringType()),
        StructField("PinCode", StringType()),
        StructField("ContactNumber", StringType())
    ])),
    StructField("InvoiceLineItems", ArrayType(StructType([
        StructField("ItemCode", StringType()),
        StructField("ItemDescription", StringType()),
        StructField("ItemPrice", DoubleType()),
        StructField("ItemQty", IntegerType()),
        StructField("TotalValue", DoubleType())
    ]))),
])

In [4]:
from pyspark.sql.functions import from_json, col
valueDF = kafkaDFS.select(from_json(col("value").cast("string"), esquema).alias("value"))
valueDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- InvoiceNumber: string (nullable = true)
 |    |-- CreatedTime: long (nullable = true)
 |    |-- StoreID: string (nullable = true)
 |    |-- PosID: string (nullable = true)
 |    |-- CashierID: string (nullable = true)
 |    |-- CustomerType: string (nullable = true)
 |    |-- CustomerCardNo: string (nullable = true)
 |    |-- TotalAmount: double (nullable = true)
 |    |-- NumberOfItems: integer (nullable = true)
 |    |-- PaymentMethod: string (nullable = true)
 |    |-- CGST: double (nullable = true)
 |    |-- SGST: double (nullable = true)
 |    |-- CESS: double (nullable = true)
 |    |-- DeliveryType: string (nullable = true)
 |    |-- DeliveryAddress: struct (nullable = true)
 |    |    |-- AddressLine: string (nullable = true)
 |    |    |-- City: string (nullable = true)
 |    |    |-- State: string (nullable = true)
 |    |    |-- PinCode: string (nullable = true)
 |    |    |-- ContactNumber: string (nullable = true)
 |    |

In [5]:
from pyspark.sql.functions import expr

explodeDF = valueDF.selectExpr("value.InvoiceNumber", "value.CreatedTime",
    "value.StoreID", "value.PosID", "value.CustomerType",
    "value.PaymentMethod", "value.DeliveryType", "value.DeliveryAddress.City",
    "value.DeliveryAddress.State", "value.DeliveryAddress.PinCode",
    "explode(value.InvoiceLineItems) as LineItem")

limpioDF = explodeDF \
    .withColumn("ItemCode", expr("LineItem.ItemCode")) \
    .withColumn("ItemDescription", expr("LineItem.ItemDescription")) \
    .withColumn("ItemPrice", expr("LineItem.ItemPrice")) \
    .withColumn("ItemQty", expr("LineItem.ItemQty")) \
    .withColumn("TotalValue", expr("LineItem.TotalValue")) \
    .drop("LineItem")

In [6]:
facturaWriterQuery  = limpioDF.writeStream \
    .format("json") \
    .queryName("Facturas Kafka Writer") \
    .outputMode("append") \
    .option("path", "salidaKafka") \
    .option("checkpointLocation", "chk-point-dir-caso3") \
    .trigger(processingTime="1 minute") \
    .start()
spark.streams.awaitAnyTermination()

25/02/20 19:35:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/02/20 19:35:59 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
     

KeyboardInterrupt: 

EJEMPLO 4

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Kafka Streaming Sink") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4") \
        .master("local[*]") \
        .getOrCreate()

kafkaDFS = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092") \
        .option("subscribe", "facturas") \
        .option("startingOffsets", "earliest") \
        .load()

25/02/20 19:43:26 WARN Utils: Your hostname, iadb-12 resolves to a loopback address: 127.0.1.1; using 172.20.104.106 instead (on interface enp45s0)
25/02/20 19:43:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/iabd/.ivy2/cache
The jars for the packages stored in: /home/iabd/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5208612f-b64e-4788-99d9-d5c825a9912f;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 257ms :: artifacts dl 19ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType
esquema = StructType([
    StructField("InvoiceNumber", StringType()),
    StructField("CreatedTime", LongType()),
    StructField("StoreID", StringType()),
    StructField("PosID", StringType()),
    StructField("CashierID", StringType()),
    StructField("CustomerType", StringType()),
    StructField("CustomerCardNo", StringType()),
    StructField("TotalAmount", DoubleType()),
    StructField("NumberOfItems", IntegerType()),
    StructField("PaymentMethod", StringType()),
    StructField("CGST", DoubleType()),
    StructField("SGST", DoubleType()),
    StructField("CESS", DoubleType()),
    StructField("DeliveryType", StringType()),
    StructField("DeliveryAddress", StructType([
        StructField("AddressLine", StringType()),
        StructField("City", StringType()),
        StructField("State", StringType()),
        StructField("PinCode", StringType()),
        StructField("ContactNumber", StringType())
    ])),
    StructField("InvoiceLineItems", ArrayType(StructType([
        StructField("ItemCode", StringType()),
        StructField("ItemDescription", StringType()),
        StructField("ItemPrice", DoubleType()),
        StructField("ItemQty", IntegerType()),
        StructField("TotalValue", DoubleType())
    ]))),
])


In [3]:
from pyspark.sql.functions import from_json, col, expr
valueDF = kafkaDFS.select(from_json(col("value").cast("string"), esquema).alias("value"))

notificationDF = valueDF.select("value.InvoiceNumber", "value.CustomerCardNo", "value.TotalAmount") \
    .withColumn("LoyaltyPoints", expr("TotalAmount * 0.2"))

# Transformamos las cuatro columnas en lo que espera Kafka, un par de (key, value)
kafkaTargetDF = notificationDF.selectExpr("InvoiceNumber as key",
        """to_json(named_struct(
        'CustomerCardNo', CustomerCardNo,
        'TotalAmount', TotalAmount,
        'EarnedLoyaltyPoints', TotalAmount * 0.2)) as value""")

In [ ]:
notificacionWriterQuery = kafkaTargetDF \
    .writeStream \
    .queryName("Notificaciones Writer") \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "notificaciones") \
    .outputMode("append") \
    .option("checkpointLocation", "chk-point-dir-caso4") \
    .start()
spark.streams.awaitAnyTermination()

25/02/20 19:43:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/02/20 19:43:29 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/pyspark/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
     

KeyboardInterrupt: 

EJERICICIO 4